In [1]:
#setting up spotipy
import sys
import spotipy
import spotipy.util as util

scope = 'user-library-read'

# if len(sys.argv) > 1:
#     username = sys.argv[1]
# else:
#     print ("Usage: %s username" % (sys.argv[0],))
#     sys.exit()
    
# token = util.prompt_for_user_token(username,scope,client_id='ff9ff248e80e428bbb9796e1a7d62aeb',
#                                    client_secret='7f70a7c884494c6c96d2e6f03c4d2388', redirect_uri='http://localhost/')

from spotipy.oauth2 import SpotifyClientCredentials

client_credentials_manager = SpotifyClientCredentials(client_id='ff9ff248e80e428bbb9796e1a7d62aeb', client_secret='7f70a7c884494c6c96d2e6f03c4d2388')
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [2]:
import pandas as pd
import numpy as np

# sample = pd.read_csv('sample.csv')

In [3]:
df = pd.read_csv('billboard_weekly_id.csv')

In [4]:
df.shape

(317795, 12)

In [17]:
df[df['sp_id'].notnull()].to_csv('billboard_with_ids.csv')

In [14]:
df[df['sp_id'].isnull()]

(26882, 12)

In [ ]:
feature_names = ['duration_ms', 'key', 'mode', 'time_signature', 'acousticness', 'danceability', 
                'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'valence', 'tempo']

In [ ]:
def get_features_with_id(id):
    track = sp.audio_features(id)[0]
    
    return [track[feature] for feature in feature_names]
    

In [ ]:
column_names = list(sample.columns)
column_names.extend(feature_names)
sample = sample.reindex(columns = column_names)

In [ ]:
sample[feature_names] = sample['Spotify_Track_ID'].apply(lambda x: pd.Series(get_features_with_id(x)))

In [ ]:
column_names = list(sample.columns)
column_names.extend(feature_names)
sample.reindex(columns = column_names)

In [ ]:
analysis = sp.audio_analysis('2XkuSbp5say8nZW8g6156Z')

In [3]:
billboard_weekly = pd.read_csv('https://query.data.world/s/fmip5pplwfadgvsbinribf5sgs7rcx')

In [24]:
def get_track_id(track_name = "", artist = ""):
    # 'featuring' problem
    if 'Featuring' in artist:
        artist = artist.split('Featuring')[0]

#     print(track_name, artist)
    
    q = track_name + ' ' + artist
    search_results = sp.search(q)
    
    #Index Error
    try:
        track_id = search_results['tracks']['items'][0]['id']
    except IndexError:
        track_id = 'null'
            
    return track_id

In [4]:
def index_marks(nrows, chunk_size):
    return range(1 * chunk_size, (nrows // chunk_size + 1) * chunk_size, chunk_size)

def split(dfm, chunk_size):
    indices = index_marks(dfm.shape[0], chunk_size)
    print(dfm.shape[0])
    return np.split(dfm, indices)

chunks = split(billboard_weekly, 5000)

317795


In [82]:
count = 0
for c in chunks:
    count += 1
    print('Chunk #: ' + str(count))
    token = util.prompt_for_user_token(username,scope,client_id='ff9ff248e80e428bbb9796e1a7d62aeb',
                                   client_secret='7f70a7c884494c6c96d2e6f03c4d2388', redirect_uri='http://localhost/')
    
    sp = spotipy.Spotify(auth=token)
    c = parallelize_dataframe(c, get_sp_id, 4)

Chunk #: 1


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

Process ForkPoolWorker-330:
Process ForkPoolWorker-331:
Traceback (most recent call last):
Process ForkPoolWorker-332:
Process ForkPoolWorker-333:
Traceback (most recent call last):
  File "/home/shenghao/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/shenghao/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/shenghao/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/home/shenghao/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/shenghao/anaconda3/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/home/shenghao/anaconda3/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/home/shenghao/anaconda3/lib/pyth

In [30]:
billboard_weekly.head()

,url,WeekID,Week Position,Song,Performer,SongID,Instance,Previous Week Position,Peak Position,Weeks on Chart
0,http://www.billboard.com/charts/hot-100/1990-0...,2/10/1990,75,Don't Wanna Fall In Love,Jane Child,Don't Wanna Fall In LoveJane Child,1,NaN,75,1
1,http://www.billboard.com/charts/hot-100/1990-0...,2/17/1990,53,Don't Wanna Fall In Love,Jane Child,Don't Wanna Fall In LoveJane Child,1,75.0,53,2
2,http://www.billboard.com/charts/hot-100/1990-0...,2/24/1990,43,Don't Wanna Fall In Love,Jane Child,Don't Wanna Fall In LoveJane Child,1,53.0,43,3
3,http://www.billboard.com/charts/hot-100/1990-0...,3/3/1990,37,Don't Wanna Fall In Love,Jane Child,Don't Wanna Fall In LoveJane Child,1,43.0,37,4
4,http://www.billboard.com/charts/hot-100/1990-0...,3/10/1990,27,Don't Wanna Fall In Love,Jane Child,Don't Wanna Fall In LoveJane Child,1,37.0,27,5


In [30]:
from pandarallel import pandarallel

pandarallel.initialize(progress_bar=True)

def get_sp_id(row):
    artist = row.Performer
    track_name = row.Song
    
    if 'Featuring' in row.Performer:
        artist = artist.split('Featuring')[0]
    
    if '&' in artist:
        artist = artist.split('&')[0]
        
    if 'With' in artist:
        artist = artist.split('With')[0]
        
#     print(track_name, artist)
    q = track_name + ' ' + artist
    search_results = sp.search(q)
    
    #Index Error
    try:
        track_id = search_results['tracks']['items'][0]['id']
    except IndexError:
        track_id = 'null'
            
    return track_id



New pandarallel memory created - Size: 2000 MB
Pandarallel will run on 8 workers


In [56]:
#test:
%time result = chunks[0].parallel_apply(get_sp_id, axis=1)


CPU times: user 2.2 s, sys: 610 ms, total: 2.81 s
Wall time: 1min 14s


In [40]:
len(result)

1000

In [6]:
count = 0
result = []
for chunk in chunks:
    count += 1
    print('Chunk #: ' + str(count))
#     sp = spotipy.Spotify(auth=token)
    
    %time chunk_result = chunk.parallel_apply(get_sp_id, axis=1)
    
    result.extend(chunk_result)


Chunk #: 1


retrying ...2secs
retrying ...2secs
retrying ...2secs
retrying ...2secs
retrying ...2secs
retrying ...2secs
retrying ...2secs
retrying ...2secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...7secs
retrying ...7secs
retrying ...7secs
retrying ...7secs
retrying ...7secs
retrying ...7secs
retrying ...7secs
retrying ...7secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...7secs
retrying ...7secs
retrying ...7secs
retrying ...7secs
retrying ...7secs
retrying ...7secs
retrying ...7secs
retrying .

Process ForkPoolWorker-3:
Process ForkPoolWorker-2:
Process ForkPoolWorker-8:
Process ForkPoolWorker-7:
Process ForkPoolWorker-4:
Process ForkPoolWorker-1:
Process ForkPoolWorker-5:
Process ForkPoolWorker-6:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/shenghao/anaconda3/lib/python3.7/site-packages/spotipy/client.py", line 119, in _internal_call
    r.raise_for_status()
  File "/home/shenghao/anaconda3/lib/python3.7/site-packages/spotipy/client.py", line 119, in _internal_call
    r.raise_for_status()
  File "/home/shenghao/anaconda3/lib/python3.7/site-packages/spotipy/client.py", line 119, in _internal_call
    r.raise_for_status()
  File "/home/shenghao/anaconda3/lib/python3.7/site-packages/spotipy/client.py", line 119, in _internal_call
 

Traceback (most recent call last):
  File "/home/shenghao/anaconda3/lib/python3.7/site-packages/spotipy/client.py", line 146, in _get
    return self._internal_call('GET', url, payload, kwargs)
spotipy.client.SpotifyException: http status: 429, code:-1 - https://api.spotify.com/v1/search?q=Don%27t+Wanna+Know+Maroon+5+&limit=10&offset=0&type=track:
 API rate limit exceeded
spotipy.client.SpotifyException: http status: 429, code:-1 - https://api.spotify.com/v1/search?q=Have+Mercy+Baby+The+Bobbettes&limit=10&offset=0&type=track:
 API rate limit exceeded
  File "/home/shenghao/anaconda3/lib/python3.7/site-packages/spotipy/client.py", line 124, in _internal_call
    headers=r.headers)
Traceback (most recent call last):
spotipy.client.SpotifyException: http status: 429, code:-1 - https://api.spotify.com/v1/search?q=Good+Girl+Carrie+Underwood&limit=10&offset=0&type=track:
 API rate limit exceeded

During handling of the above exception, another exception occurred:

  File "/home/shenghao/anac

  File "/home/shenghao/anaconda3/lib/python3.7/site-packages/pandas/core/apply.py", line 251, in apply_standard
    labels=labels)
  File "/home/shenghao/anaconda3/lib/python3.7/site-packages/pandarallel/dataframe.py", line 43, in worker_apply
    res = df[axis_chunk].apply(func_to_apply, *args, **kwargs)
  File "/home/shenghao/anaconda3/lib/python3.7/site-packages/multiprocess/pool.py", line 44, in mapstar
    return list(map(*args))
  File "/home/shenghao/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py", line 6487, in apply
    return op.get_result()
  File "/home/shenghao/anaconda3/lib/python3.7/site-packages/pathos/helpers/mp_helper.py", line 15, in <lambda>
    func = lambda args: f(*args)
  File "/home/shenghao/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py", line 6487, in apply
    return op.get_result()
  File "/home/shenghao/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py", line 6487, in apply
    return op.get_result()
  File "pandas/_libs/re

  File "/home/shenghao/anaconda3/lib/python3.7/site-packages/spotipy/client.py", line 157, in _get
    time.sleep(sleep_seconds)
  File "/home/shenghao/anaconda3/lib/python3.7/site-packages/spotipy/client.py", line 157, in _get
    time.sleep(sleep_seconds)
KeyboardInterrupt
  File "/home/shenghao/anaconda3/lib/python3.7/site-packages/spotipy/client.py", line 157, in _get
    time.sleep(sleep_seconds)
KeyboardInterrupt
  File "/home/shenghao/anaconda3/lib/python3.7/site-packages/pandarallel/dataframe.py", line 35, in decorator
    return func(*args, **kwargs)
KeyboardInterrupt
  File "<ipython-input-5-8aab3166de7d>", line 17, in get_sp_id
    search_results = sp.search(q)
KeyboardInterrupt
  File "/home/shenghao/anaconda3/lib/python3.7/site-packages/spotipy/client.py", line 157, in _get
    time.sleep(sleep_seconds)
  File "/home/shenghao/anaconda3/lib/python3.7/site-packages/spotipy/client.py", line 339, in search
    return self._get('search', q=q, limit=limit, offset=offset, type=ty

ArrowIOError: Broken pipe

NameError: name 'chunk_result' is not defined

In [6]:
%time id_list = billboard_weekly.parallel_apply(get_sp_id, axis=1)

retrying ...7secs
retrying ...7secs
retrying ...7secs
retrying ...7secs
retrying ...7secs
retrying ...7secs
retrying ...7secs
retrying ...7secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...5secs
retrying ...1secs
retrying ...1secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...5secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying .

retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...5secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...7secs
retrying ...7secs
retrying ...7secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...1secs
retrying ...1secs
retrying .

retrying ...7secs
retrying ...7secs
retrying ...7secs
retrying ...7secs
retrying ...7secs
retrying ...7secs
retrying ...7secs
retrying ...7secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...7secs
retrying ...7secs
retrying ...7secs
retrying ...7secs
retrying ...7secs
retrying ...7secs
retrying ...6secs
retrying ...6secs
retrying ...1secs
retrying ...1secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...5secs
retrying ...5secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...5secs
retrying ...5secs
retrying ...5secs
retrying ...5secs
retrying ...5secs
retrying ...5secs
retrying ...5secs
retrying ...5secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...7secs
retrying ...6secs
retrying .

retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...5secs
retrying ...5secs
retrying ...5secs
retrying ...5secs
retrying ...5secs
retrying ...5secs
retrying ...5secs
retrying ...4secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...6secs
retrying ...5secs
retrying ...5secs
retrying ...5secs
retrying ...5secs
retrying ...5secs
retrying ...4secs
retrying ...4secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying .

retrying ...1secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...7secs
retrying ...7secs
retrying ...7secs
retrying ...7secs
retrying ...7secs
retrying ...7secs
retrying ...7secs
retrying ...7secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...1secs
retrying ...1secs
retrying .

...

retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...5secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...4secs
retrying ...4secs
retrying ...4secs
retrying ...4secs
retrying ...4secs
retrying ...4secs
retrying ...3secs
retrying ...3secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...2secs
retrying ...2secs
retrying ...2secs
retrying ...2secs
retrying .

retrying ...2secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...4secs
retrying ...4secs
retrying ...4secs
retrying ...4secs
retrying ...4secs
retrying ...4secs
retrying ...4secs
retrying ...3secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying .

retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...7secs
retrying ...7secs
retrying ...7secs
retrying ...7secs
retrying ...7secs
retrying ...7secs
retrying ...7secs
retrying ...7secs
retrying ...7secs
retrying ...7secs
retrying ...7secs
retrying ...7secs
retrying ...7secs
retrying ...7secs
retrying ...7secs
retrying ...7secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...5secs
retrying ...5secs
retrying ...5secs
retrying ...5secs
retrying ...5secs
retrying ...5secs
retrying ...4secs
retrying ...4secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...5secs
retrying ...5secs
retrying ...5secs
retrying ...5secs
retrying ...5secs
retrying ...5secs
retrying ...5secs
retrying ...4secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying .

retrying ...1secs
retrying ...7secs
retrying ...7secs
retrying ...7secs
retrying ...7secs
retrying ...7secs
retrying ...7secs
retrying ...7secs
retrying ...7secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...4secs
retrying ...4secs
retrying ...4secs
retrying ...4secs
retrying ...4secs
retrying ...4secs
retrying ...4secs
retrying ...4secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...4secs
retrying ...4secs
retrying ...4secs
retrying ...4secs
retrying ...4secs
retrying ...4secs
retrying ...4secs
retrying ...4secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...4secs
retrying ...4secs
retrying ...4secs
retrying ...4secs
retrying ...4secs
retrying ...1secs
retrying ...1secs
retrying .

retrying ...4secs
retrying ...4secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...5secs
retrying ...5secs
retrying ...5secs
retrying ...5secs
retrying ...5secs
retrying ...5secs
retrying ...5secs
retrying ...4secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...5secs
retrying ...4secs
retrying ...4secs
retrying ...4secs
retrying ...4secs
retrying ...4secs
retrying ...4secs
retrying ...4secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...5secs
retrying ...5secs
retrying ...5secs
retrying ...5secs
retrying ...5secs
retrying ...5secs
retrying ...4secs
retrying ...4secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...5secs
retrying .

In [8]:
len(id_list)

317795

In [9]:
billboard_weekly.shape

(317795, 10)

In [10]:
billboard_weekly['sp_id'] = id_list

In [11]:
billboard_weekly.to_csv('billboard_weekly_id.csv')

In [12]:
sum(id_list == 'null')

26882

In [29]:
null_df = billboard_weekly[billboard_weekly['sp_id'] == 'null']

In [28]:
sp.search('Return To Sender')

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=Return+To+Sender&type=track&offset=0&limit=10',
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/43ZHCT0cAZBISjO8DG9PnE'},
       'href': 'https://api.spotify.com/v1/artists/43ZHCT0cAZBISjO8DG9PnE',
       'id': '43ZHCT0cAZBISjO8DG9PnE',
       'name': 'Elvis Presley',
       'type': 'artist',
       'uri': 'spotify:artist:43ZHCT0cAZBISjO8DG9PnE'}],
     'available_markets': ['AD',
      'AE',
      'AR',
      'AT',
      'AU',
      'BE',
      'BG',
      'BH',
      'BO',
      'BR',
      'CA',
      'CH',
      'CL',
      'CO',
      'CR',
      'CY',
      'CZ',
      'DE',
      'DK',
      'DO',
      'DZ',
      'EC',
      'EE',
      'EG',
      'ES',
      'FI',
      'FR',
      'GB',
      'GR',
      'GT',
      'HK',
      'HN',
      'HU',
      'ID',
      'IE',
      'IL',
      'IN',
      'IS',
      'IT',
      'JO',
      'JP

In [31]:
%time null_id_list = null_df.parallel_apply(get_sp_id, axis=1)

CPU times: user 56.9 s, sys: 7.43 s, total: 1min 4s
Wall time: 28min 51s


In [33]:
sum(null_id_list=='null')

25531

In [41]:
null_id_list.head()

128    null
129    null
130    null
131    null
132    null
dtype: object

In [43]:
null_df[null_df['sp_id'] != 'null']

,url,WeekID,Week Position,Song,Performer,SongID,Instance,Previous Week Position,Peak Position,Weeks on Chart,sp_id
423,http://www.billboard.com/charts/hot-100/1958-1...,12/6/1958,99,Don't You Know Yockomo,"Huey ""piano"" Smith With His Clowns","Don't You Know YockomoHuey ""piano"" Smith With ...",1,NaN,99,1,65nJk1OAuMHYRbiUbnr4Ug
424,http://www.billboard.com/charts/hot-100/1959-0...,1/3/1959,71,Don't You Know Yockomo,"Huey ""piano"" Smith With His Clowns","Don't You Know YockomoHuey ""piano"" Smith With ...",2,NaN,71,2,65nJk1OAuMHYRbiUbnr4Ug
425,http://www.billboard.com/charts/hot-100/1959-0...,1/10/1959,63,Don't You Know Yockomo,"Huey ""piano"" Smith With His Clowns","Don't You Know YockomoHuey ""piano"" Smith With ...",2,71.0,63,3,65nJk1OAuMHYRbiUbnr4Ug
426,http://www.billboard.com/charts/hot-100/1959-0...,1/17/1959,58,Don't You Know Yockomo,"Huey ""piano"" Smith With His Clowns","Don't You Know YockomoHuey ""piano"" Smith With ...",2,63.0,58,4,65nJk1OAuMHYRbiUbnr4Ug
427,http://www.billboard.com/charts/hot-100/1959-0...,1/24/1959,56,Don't You Know Yockomo,"Huey ""piano"" Smith With His Clowns","Don't You Know YockomoHuey ""piano"" Smith With ...",2,58.0,56,5,65nJk1OAuMHYRbiUbnr4Ug
428,http://www.billboard.com/charts/hot-100/1959-0...,1/31/1959,82,Don't You Know Yockomo,"Huey ""piano"" Smith With His Clowns","Don't You Know YockomoHuey ""piano"" Smith With ...",2,56.0,56,6,65nJk1OAuMHYRbiUbnr4Ug
719,http://www.billboard.com/charts/hot-100/1962-0...,9/22/1962,10,She's Not You,Elvis Presley With The Jordanaires,She's Not YouElvis Presley With The Jordanaires,1,6.0,5,8,7ntXUTgbm0PHd68xlS7QOC
1187,http://www.billboard.com/charts/hot-100/1960-0...,6/4/1960,30,Dutchman's Gold,Walter Brennan With Billy Vaughn and his Orche...,Dutchman's GoldWalter Brennan With Billy Vaugh...,1,31.0,30,7,6pmq7owNhitzLQp3CGYxVB
1682,http://www.billboard.com/charts/hot-100/1961-0...,2/18/1961,24,Surrender,Elvis Presley With The Jordanaires,SurrenderElvis Presley With The Jordanaires,1,NaN,24,1,366wRPrx2airjcCRkxgoS2
1683,http://www.billboard.com/charts/hot-100/1992-0...,5/30/1992,24,The Best Things In Life Are Free,Luther Vandross And Janet Jackson With BBD And...,The Best Things In Life Are FreeLuther Vandros...,1,NaN,24,1,3DVFzlAiUi4CcJRrsfKQnj


In [37]:
null_df['sp_id'] = null_id_list

/home/shenghao/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [ ]:
df = pd.merge(billboard_weekly, null_df, how='left', on='sp_id')

In [ ]:
null_id_list